<a href="https://colab.research.google.com/github/AshwinKotgire/Play_with_LLMs/blob/main/Prompt_Engineering/Rough_script_of_LLM_with_context.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##LLM( Llama v2 ) with pdf context
In previous 2 notebooks we have individually loaded Llama 7b and FAISS vectorstore . In this we combine both for incontext Q and A

##Installing required libraries
1. transformers: Hugging face libraries for loading pretrained transformer checkpoints.
2. accelerate : Manages communications between CPU and GPU more efficiently.
3.datasets : For loading datasets from huggingface for future fine tuning.
4. bitsandbytes : Used for quantization of model to run on limited resources.
5. einops :Einstein-Inspired Notation for operations
6. wandb :weights and biases for visualizations
7. Langchain : High level library assisting in creation and deployment of LLM apps : https://python.langchain.com/docs/get_started
8. sentence_transformers : SentenceTransformers 🤗 is a Python framework for state-of-the-art sentence, text and image embeddings. : https://huggingface.co/sentence-transformers
9. faiss-cpu / faiss-gpu :Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. : https://python.langchain.com/docs/integrations/vectorstores/faiss
10. pypdf : pypdf into array of documents, where each document contains the page content and metadata with page number.: https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf
11. tiktoken : Used for tokenwise splitting of string or steam . Tbn that tokenization takes place according to tokens not words.





In [1]:
!pip install -q -U trl transformers accelerate
!pip install -q datasets bitsandbytes einops wandb
!pip install langchain
!pip install sentence_transformers
# !pip install faiss-cpu
!pip install faiss-gpu
!pip install pypdf
!pip install  tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import login
login()

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import pipeline
import transformers
import torch


#####pdftextloader : https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf

In [4]:

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredURLLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.document_loaders import PDFMinerPDFasHTMLLoader
from langchain.document_loaders import PDFPlumberLoader
from langchain.text_splitter import TokenTextSplitter

from IPython.display import HTML

from langchain.document_loaders import OnlinePDFLoader


from langchain.text_splitter import SentenceTransformersTokenTextSplitter



In [5]:

model_name = "meta-llama/Llama-2-7b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
from IPython.display import Markdown
if torch.cuda.is_available():
    print('cuda')
    device = torch.device("cuda")  # If GPU is available, use it.
else:
    device = torch.device("cpu")   # If GPU is not available, use the CPU.



cuda


Observations :
1. Sometimes the 404 Error occurs where the link to the urls is not accessible even if url is correct . In such case restarting the runtime helped.
2. I observed that formating of pypdf was the best . but the rest of the loaders could also be used.
PDF loaders : https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf

##Part 1 : Going Sequentially
#### Loading Model --> Loading Embeddings and Vector Store --> Loading Context --> Splitting ,embedding and saving context --> Passing instruction and finding nearest context --> Creating Pormpt --> Passing prompt to LLM.

In [7]:
urls="https://arxiv.org/pdf/1706.03762.pdf"
loader=PyPDFLoader(urls)
data=loader.load()

In [74]:
type(data[0])

langchain.schema.document.Document

In [ ]:
content=' '
for i in data : content+=i.page_content
content

In [8]:
splitter=SentenceTransformersTokenTextSplitter(chunk_overlap=50,tokens_per_chunk=100)

ss=splitter.split_documents(data)

In [25]:

text_splitter = TokenTextSplitter(chunk_size=100, chunk_overlap=50)

texts = text_splitter.create_documents([content])


In [24]:
len(texts)

211

In [9]:
embeddings = HuggingFaceEmbeddings()

In [10]:
db = FAISS.from_documents(ss, embeddings)

In [11]:
question="What are the parts of transformers?"
contexts=db.similarity_search(question,k=2)
print(contexts)

[Document(page_content='figure 1 : the transformer - model architecture. the transformer follows this overall architecture using stacked self - attention and point - wise, fully connected layers for both the encoder and decoder, shown in the left and right halves of figure 1, respectively. 3. 1 encoder and decoder stacks encoder : the encoder is composed of a stack of n = 6 identical layers. each layer has two sub - layers. the first is a multi - head self -', metadata={'source': '/tmp/tmpxnt8yjo2/tmp.pdf', 'page': 2}), Document(page_content='. in this work we employ h = 8 parallel attention layers, or heads. for each of these we use dk = dv = dmodel / h = 64. due to the reduced dimension of each head, the total computational cost is similar to that of single - head attention with full dimensionality. 3. 2. 3 applications of attention in our model the transformer uses multi - head attention in three different ways : • in " encoder - decoder attention " layers,', metadata={'source': '/t

Input prompt for Llama is of the form :
```
prompt="""<s> [INST]
<<SYS>>
System guiding message
<</SYS>>
[/INST]"""
```
Sources:
1. https://huggingface.co/blog/llama2
2. https://discuss.huggingface.co/t/llama-2-7b-hf-repeats-context-of-question-directly-from-input-prompt-cuts-off-with-newlines/48250/10  
If not done so will result in the language model misbehaving . Therefore prompt engineering is important.

In [17]:
context=' '
for i in contexts:context+=i.page_content
context

' figure 1 : the transformer - model architecture. the transformer follows this overall architecture using stacked self - attention and point - wise, fully connected layers for both the encoder and decoder, shown in the left and right halves of figure 1, respectively. 3. 1 encoder and decoder stacks encoder : the encoder is composed of a stack of n = 6 identical layers. each layer has two sub - layers. the first is a multi - head self -. in this work we employ h = 8 parallel attention layers, or heads. for each of these we use dk = dv = dmodel / h = 64. due to the reduced dimension of each head, the total computational cost is similar to that of single - head attention with full dimensionality. 3. 2. 3 applications of attention in our model the transformer uses multi - head attention in three different ways : • in " encoder - decoder attention " layers,'

In [18]:
prompt=f"""<s>[INST] <<SYS>>
You are an honest assistant , who gives factually correct answers while refering to the context. If the context does not have an answer you try your best to answer the question on your own.\n
Answer the following question while refering the context.If the answer is not in context ,try to answer on your own . Still if you are not able to answer the question then politely say so instead of ramblling.\n
<</SYS>>
Context:{context}
Qustion:{question}
 [/INST]
"""
print(prompt)

<s>[INST] <<SYS>>
You are an honest assistant , who gives factually correct answers while refering to the context. If the context does not have an answer you try your best to answer the question on your own.

Answer the following question while refering the context.If the answer is not in context ,try to answer on your own . Still if you are not able to answer the question then politely say so instead of ramblling.

<</SYS>>
Context: figure 1 : the transformer - model architecture. the transformer follows this overall architecture using stacked self - attention and point - wise, fully connected layers for both the encoder and decoder, shown in the left and right halves of figure 1, respectively. 3. 1 encoder and decoder stacks encoder : the encoder is composed of a stack of n = 6 identical layers. each layer has two sub - layers. the first is a multi - head self -. in this work we employ h = 8 parallel attention layers, or heads. for each of these we use dk = dv = dmodel / h = 64. due 

In [27]:
from IPython.display import Markdown

In [ ]:
# prompt = 'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?'
inputs = tokenizer(prompt, return_tensors="pt",padding =True).to(device)
generate_ids = model.generate(inputs.input_ids, max_length=5000,top_k=1,top_p=0.5,temperature=0)


Skipping the prompt from the result.

In [ ]:
p=generate_ids[0][inputs['input_ids'].shape[1]:]

Markdown for better outputs.
Results are quite astonishing for a 7b parameter model.

In [ ]:
Markdown(tokenizer.decode(p, skip_special_tokens=True, clean_up_tokenization_spaces=True))
# print(tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])

##Part 2 : Creating Chain using Langchain

In [14]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


In [40]:
from langchain import HuggingFacePipeline


In [12]:
prompt="""<s>[INST] <<SYS>>
You are an honest assistant , who gives factually correct answers while refering to the context. If the context does not have an answer you try your best to answer the question on your own.\n
Answer the following question while refering the context.If the answer is not in context ,try to answer on your own . Still if you are not able to answer the question then politely say so instead of ramblling.\n
<</SYS>>
Context:{context}
Qustion:{question}
 [/INST]
"""

In [15]:
prompt_t=PromptTemplate(input_variables=['context','question'],template=prompt)
prompt_t

PromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, template='<s>[INST] <<SYS>>\nYou are an honest assistant , who gives factually correct answers while refering to the context. If the context does not have an answer you try your best to answer the question on your own.\n\nAnswer the following question while refering the context.If the answer is not in context ,try to answer on your own . Still if you are not able to answer the question then politely say so instead of ramblling.\n\n<</SYS>>\nContext:{context}\nQustion:{question}\n [/INST]\n', template_format='f-string', validate_template=True)

In [19]:
pipe=pipeline(task='text-generation',model=model,tokenizer=tokenizer)


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
outputs = pipe(prompt, num_return_sequences=1, return_full_text=False)
outputs
Markdown(outputs[0]['generated_text'])

In [41]:
local_llm=HuggingFacePipeline(pipeline=pipe)

In [43]:
chain=LLMChain(llm=local_llm,prompt=prompt_t)

In [44]:
chain.predict(context=context,question=question)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (4096) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


"Based on the context you provided, the parts of the Transformer model are:\n\n1. Encoder: The encoder is composed of a stack of $n$ identical layers, where each layer has two sub-layers. The first sub-layer is a multi-head self-attention layer, and the second sub-layer is a point-wise, fully connected layer.\n2. Decoder: The decoder is also composed of a stack of $n$ identical layers, where each layer has two sub-layers. The first sub-layer is a multi-head self-attention layer, and the second sub-layer is a point-wise, fully connected layer.\n3. Encoder-decoder attention: The Transformer uses multi-head attention in this way, where the encoder and decoder attend to each other's output.\n\nPlease let me know if you have any further questions!"

In [80]:
class Cust_Chain_obj():
  def __init__(self,model,FAISS_obj,sys_prompt='',embeddings = HuggingFaceEmbeddings(),text_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=50)):
    self.FAISS_obj=FAISS_obj
    self.model=model
    self.sys_prompt=sys_prompt
    self.embedding_obj=embeddings
    self.text_splitter_obj=text_splitter
    self.doc=[]
    self.prompt_template=self.create_prompt_template()
    self.LLM_Chain=self.create_chain()
  def create_prompt_template(self):
    prompt="""<s>[INST] """+self.sys_prompt+"""
        Context:{context}
        Qustion:{question}
        [/INST]
        """
    prompt_t=PromptTemplate(input_variables=['context','question'],template=prompt)
    return prompt_t

  def load_pdf_doc(self,doc_path):
    loader=PyPDFLoader(urls)
    data=loader.load()
    self.doc=data
    return self.doc[:]

  def set_embedding_obj(self,new_embedding_obj):
    self.embedding_obj=new_embedding_obj

  def set_sys_prompt(self,new_sys_prompt):
    self.sys_prompt=f"<<SYS>>{new_sys_prompt}<</SYS>>"

  def set_new_model(self,new_model):
    self.model=new_model

  def populate_vector_store(self,document,embedding_obj=None,text_splitter=None):
    if embedding_obj is None:
      embedding_obj=self.embedding_obj
    if (text_splitter is None):
      text_splitter=self.text_splitter_obj
    docs=text_splitter.split_documents(document)
    self.FAISS_obj=FAISS.from_documents(docs,embedding_obj)

  def retrieve_documents(self,query,k=2):
    contexts=self.FAISS_obj.similarity_search(query,k=k)
    return contexts

  def retrieve_contexts(self,query,k=2,use_contexts=False)  :
    context = "No context available answer on your own.\n"
    meta_datas=[]
    if(use_contexts==True):
      contexts=self.retrieve_documents(query,k)
      cc=''
      for c in contexts:
        cc+=c.page_contents
        meta_datas.append(c.meta_data)
        cc+='\n'
      if(len(cc)!=0  ):
        context=cc
    return context,meta_datas
  def create_chain(self):
    return LLMChain(llm=self.model,prompt=self.prompt_t)







In [81]:
chain=Cust_Chain_obj(None,None,"None")